In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import numpy as np
import matplotlib.dates as mdates
import datetime
#sns.set(color_codes=True)
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42
import statistics as st
sns.set_style('whitegrid', {'axes.linewidth' : 0.5})
from statsmodels.distributions.empirical_distribution import ECDF
import scipy
import gc
import collections

import sys
sys.path.append('../prepare_gustav/src/')

from gustav import ebi, ncbi, nlm, nih
from gustav import publications
from gustav import github
from gustav import access_framework
from gtfparse import read_gtf

In [2]:
from bokeh.io import output_notebook, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine, Circle, EdgesAndLinkedNodes, TapTool, NodesAndLinkedEdges, OpenURL, Title
from bokeh.plotting import figure
from bokeh.plotting import from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap

In [3]:
output_notebook()

Loading BokehJS ...

In [4]:
import networkx as nx

In [39]:
%%time
gene_info = ncbi.gene_info(taxon_ncbi=9606, mode='unambiguous_ensembl')

gene2pubmed = ncbi.gene2pubmed(taxon_ncbi=9606)


ml = ncbi.pubmed(
    'main',
    ['pubmed_id', 'pubdate']   # there are many more columns, including abstracts
).rename(
    columns={'pubdate': 'year'}
).groupby('pubmed_id')['year'].min().reset_index()

gene2pubmed = pd.merge(gene2pubmed, ml, on='pubmed_id', how='left')

gene2pubmed_new = gene2pubmed.copy()

gene2pubmed_new = gene2pubmed_new[(gene2pubmed_new['year'] >= 2000) & (gene2pubmed_new['year'] <= 2018)]

Wall time: 23.5 s


In [8]:
mesh = pd.read_parquet('data/intersect_pubmed_ids_to_mesh_terms.parquet')

In [73]:
pn_df = pd.read_csv('./proteostasis/pn_network_211212.csv')

name_to_entrez_dict = dict(zip(gene_info['symbol_ncbi'], gene_info['gene_ncbi']))

# get entrez for genes with only name
pn_df['gene_ncbi'] = pn_df['symbol_ncbi'].apply(lambda x: name_to_entrez_dict.get(x))

# drop those still without entrez
pn_df = pn_df.dropna(subset=['gene_ncbi'])

# insert Falses
pn_df[['cytosol','endoplasmic_reticulum', 'mitochondrial', 'ubq', 'autophagy',
       'ribosomal_qc', 'canon_chaperones', 'ambiguous']] = pn_df.loc[:, ['cytosol',
       'endoplasmic_reticulum', 'mitochondrial', 'ubq', 'autophagy',
       'ribosomal_qc', 'canon_chaperones', 'ambiguous']].fillna(False)

# remove ambiguous
#pn_df = pn_df[~pn_df['ambiguous']]

# remove duplicates
pn_df = pn_df.groupby('gene_entrez').max().reset_index()

In [74]:
# neat str describing pn_role
pn_df['pn_role'] = ''

for index, row in pn_df.iterrows():
    
    pn_role = ''
    if row['canon_chaperones']:
        pn_role = pn_role + 'canon chaperone, '
    if row['cytosol']:
        pn_role = pn_role + 'cytosolic, '
    if row['endoplasmic_reticulum']:
        pn_role = pn_role + 'ER, '
    if row['mitochondrial']:
        pn_role = pn_role + 'ER, '
    if row['ubq']:
        pn_role = pn_role + 'ubiquitin system, '
    if row['autophagy']:
        pn_role = pn_role + 'autophagy, '
    if row['ribosomal_qc']:
        pn_role = pn_role + 'ribosome QC, '
    if row['ambiguous']:
        pn_role = pn_role + ' ambiguous, '
        
    if np.sum(row[['cytosol','endoplasmic_reticulum', 'mitochondrial', 'ubq', 'autophagy',
       'ribosomal_qc', 'canon_chaperones']]) > 0:
        pn_role = pn_role[:-2]
        
    pn_df.loc[index, 'pn_role'] = pn_role

In [75]:
def get_partner_str(node, n_partners = 3):
    '''
        returns top-weighted node partners. node is a int index.
    '''
    partners_str = ''
    weights = np.array([edge[2].get('weight') for edge in list(G.edges(node, data=True))])
    partners = np.array([edge[1] for edge in list(G.edges(node, data=True))])
    top3_weights = weights[(-weights).argsort()][0:n_partners]
    top3_partners = partners[(-weights).argsort()][0:n_partners]
    for weight, partner in zip(top3_weights, top3_partners):
        partner_symbol = gene_info.loc[gene_info['gene_ncbi'] == partner]['symbol_ncbi'].values[0]
        partners_str = partners_str + partner_symbol + ' (' + str(weight) + ' pubs), '
    partners_str = partners_str[:-2]
    return partners_str

In [76]:
%%time

palette = ['blue', 'red']
valid_pubmed_ids = gene2pubmed_new['pubmed_id'].value_counts()

valid_pubmed_ids = valid_pubmed_ids[(valid_pubmed_ids > 1) & (valid_pubmed_ids <= 10)]

gene2pubmed_new = gene2pubmed_new[gene2pubmed_new['pubmed_id'].isin(valid_pubmed_ids.index)]

gene_count = gene2pubmed[(gene2pubmed['year'] >= 2000) & (gene2pubmed['year'] <= 2018)]['gene_ncbi'].value_counts()

genes_of_interest = gene_count[(gene_count.index.isin(gene_info['gene_ncbi'])) & 
                               (gene_count.index.isin(pn_df['gene_ncbi']))].index.values

gene2pubmed_unstack = gene2pubmed_new[gene2pubmed_new['gene_ncbi'].isin(genes_of_interest)].groupby(['pubmed_id', 'gene_ncbi']).size().unstack()
gene2pubmed_unstack = gene2pubmed_unstack.fillna(0).astype(int)

co_occ = gene2pubmed_unstack.T.dot(gene2pubmed_unstack)
co_occ.values[tuple([np.arange(co_occ.shape[0])]*2)] = 0
G = nx.from_pandas_adjacency(co_occ)

# number of publications attribute
n_pubs = dict([(node, gene_count[node]) for node in list(G)])
nx.set_node_attributes(G, name='n_pubs', values=n_pubs)

# node size attribute
n_pubs_to_size = dict([(node, 1*np.sqrt(gene_count[node])) for node in list(G)])
nx.set_node_attributes(G, name='size', values=n_pubs_to_size)

# gene symbol attribute
symbols = dict([(node, gene_info.loc[gene_info['gene_ncbi'] == node]['symbol_ncbi'].values[0]) for node in list(G)])
nx.set_node_attributes(G, name='symbol', values=symbols)

# gene description attribute
full_names = dict([(node, gene_info.loc[gene_info['gene_ncbi'] == node]['description'].values[0]) for node in list(G)])
nx.set_node_attributes(G, name='description', values=full_names)

# role in proteostasis
pn_role = dict([(node, pn_df.loc[pn_df['gene_ncbi'] == node]['pn_role'].values[0]) for node in list(G)])
nx.set_node_attributes(G, name='pn_role', values=pn_role)

# ambiguous
ambig = dict([(node, palette[int(pn_df.loc[pn_df['gene_ncbi'] == node]['ambiguous'].values[0])]) for node in list(G)])
nx.set_node_attributes(G, name='ambig', values=ambig)

# partner list
partner_strs = dict([(node, get_partner_str(node, n_partners=3)) for node in list(G)])
nx.set_node_attributes(G, name='top_partners', values=partner_strs)

# weighted degree
degrees = dict(nx.degree(G, weight='weight'))
nx.set_node_attributes(G, name='w_degree', values=degrees)

# unweighted degree
degrees = dict(nx.degree(G))
nx.set_node_attributes(G, name='u_degree', values=degrees)

# betweeness centrality
bc = dict(nx.betweenness_centrality(G, weight='weight'))
nx.set_node_attributes(G, name='betweenness_centrality', values=bc)

# website
site = dict([(node, 'ncbi.nlm.nih.gov/gene/' + str(node)) for node in list(G)])
nx.set_node_attributes(G, name='NCBI', values=site)

Wall time: 3.99 s


In [79]:
# seed for node positions
spring_seed=0

# size and color attributes
size_by_this_attribute = 'size'
color_by_this_attribute = 'ambig'

#Pick a color palette — Blues8, Reds8, Purples8, Oranges8, Viridis8
color_palette = Spectral8

#Choose colors for node and edge highlighting
node_highlight_color = 'red'
edge_highlight_color = 'red'

title = 'Gene literature network (proteostasis network, 429 genes)'

#Establish which categories will appear when hovering over each node
hover_tooltips = [
    ("EntrezID", "@index"),
    ("gene symbol", "@symbol"),
    ("gene name", "@description"),
    ("NCBI (click node to open)", "@NCBI"), 
    ("role", "@pn_role"),
    ("# pubs (2000-2018)", "@n_pubs"),
    ("# co-mentioned genes (in this net)", "@u_degree"),
    ("# co-mentions (in this net)", "@w_degree"), 
    ("top partners", "@top_partners")
]

#Create a plot — set dimensions, toolbar, and title
plot = figure(tooltips = hover_tooltips,
              tools="pan,wheel_zoom,save,reset,tap", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title, plot_width=1000, plot_height=600)

# add subtitles
plot.add_layout(Title(text="Node size corresponds to # publications from 2000 to 2018, edge width corresponds to # co-mentions of two genes", text_font_style="italic"), 'above')
plot.add_layout(Title(text="Node color represents TPR-domain proteins with no known proteostasis function", text_font_style="italic"), 'above')
plot.add_layout(Title(text="Hover for gene info, click to open NCBI page", text_font_style="italic"), 'below')

#Create a network graph object
network_graph = from_networkx(G, nx.spring_layout, scale=10, center=(0, 0), k=2, seed=spring_seed)

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
maximum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
network_graph.node_renderer.glyph = Circle(size=size_by_this_attribute, 
                                           fill_color=color_by_this_attribute)
network_graph.node_renderer.hover_glyph = Circle(size=size_by_this_attribute, 
                                                 fill_color=node_highlight_color, 
                                                 line_width=2)
network_graph.node_renderer.selection_glyph = Circle(size=size_by_this_attribute, 
                                                     fill_color=node_highlight_color, 
                                                     line_width=2)

#Set edge opacity and width
network_graph.edge_renderer.data_source.data["line_width"] = [0.1*G.get_edge_data(a,b)['weight'] for a, b in G.edges()]
network_graph.edge_renderer.data_source.data["line_alpha"] = [0.75 for edge in G.edges()]
network_graph.edge_renderer.glyph.line_width = {'field': 'line_width'}
network_graph.edge_renderer.glyph.line_alpha = {'field': 'line_alpha'}
network_graph.edge_renderer.selection_glyph = MultiLine(line_color=edge_highlight_color, line_width='line_width', line_alpha=1)
network_graph.edge_renderer.nonselection_glyph = MultiLine(line_alpha=0)
network_graph.edge_renderer.hover_glyph = MultiLine(line_color=edge_highlight_color, line_width='line_width', line_alpha=1)

network_graph.selection_policy = NodesAndLinkedEdges()
network_graph.inspection_policy = NodesAndLinkedEdges()

plot.renderers.append(network_graph)

url = "https://ncbi.nlm.nih.gov/gene/@index"
taptool = plot.select(type=TapTool)
taptool.callback = OpenURL(url=url)

plot.xgrid.grid_line_color = None
plot.ygrid.grid_line_color = None
plot.axis.visible = False

show(plot)
save(plot, filename=f"gene_literature_network_proteostasis_ambiguous.html")

C:\ProgramData\Anaconda3\lib\site-packages\bokeh\io\saving.py:127: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
C:\ProgramData\Anaconda3\lib\site-packages\bokeh\io\saving.py:140: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'C:\\Users\\Ibis Grad\\Documents\\GitHub\\gene_literature_network\\gene_literature_network_proteostasis_ambiguous.html'

In [49]:
pn_df['ambiguous']

0      False
1      False
2      False
3      False
4      False
       ...  
533     True
534    False
535    False
536    False
537    False
Name: ambiguous, Length: 538, dtype: bool

In [63]:
G[727851]

AtlasView({729857: {'weight': 1}, 729540: {'weight': 1}, 653489: {'weight': 1}, 400966: {'weight': 1}, 285190: {'weight': 1}})

In [65]:
nx.get_node_attributes(G, 'ambig')

{139322: 0,
 64427: 1,
 64175: 0,
 100652748: 0,
 100287932: 0,
 100188893: 0,
 730262: 0,
 729857: 1,
 729540: 1,
 727851: 1,
 653505: 0,
 653489: 1,
 440574: 0,
 439996: 1,
 414061: 0,
 402682: 0,
 401505: 0,
 400966: 1,
 387990: 0,
 285190: 1,
 283208: 0,
 259217: 0,
 254042: 0,
 204474: 0,
 201595: 0,
 200185: 0,
 196294: 0,
 166379: 0,
 165721: 0,
 161582: 0,
 160418: 1,
 160335: 1,
 158219: 1,
 155465: 0,
 150737: 1,
 150274: 0,
 148014: 1,
 146862: 0,
 145567: 1,
 143941: 1,
 143630: 0,
 139212: 0,
 134492: 0,
 134266: 0,
 134218: 0,
 133418: 0,
 133022: 0,
 131474: 0,
 131118: 0,
 126393: 0,
 125988: 0,
 123016: 1,
 121506: 0,
 120526: 0,
 116835: 0,
 115209: 0,
 96459: 0,
 94086: 0,
 92609: 0,
 91875: 1,
 91319: 0,
 90850: 0,
 90701: 0,
 90580: 0,
 85479: 0,
 85461: 1,
 85406: 0,
 84955: 0,
 84899: 1,
 84896: 0,
 84749: 0,
 84447: 0,
 84303: 0,
 84277: 0,
 84164: 0,
 84061: 0,
 83943: 0,
 83857: 1,
 81572: 0,
 81567: 0,
 81562: 0,
 81542: 0,
 80755: 0,
 80331: 0,
 80279: 0,
 8